Import libraries

In [2]:
import numpy as np
import math
np.set_printoptions(suppress=True, linewidth=np.nan)


# Dantzig's simplex algorithm

### Utilities

In [3]:
def build_tableau(A: np.ndarray, b: np.ndarray, c: np.ndarray) -> np.ndarray:
    tableau = np.r_[[c], A]
    tableau = np.c_[tableau, np.insert(b, 0, 0)]
    return tableau


In [4]:
def minimum_ratio_test(col: np.ndarray, b: np.ndarray) -> int:
    r_min = 0
    min_val = math.inf
    for k in range(len(b)):
        if col[k] > 0:
            if min_val > (min_val_temp := b[k] / col[k]):
                min_val = min_val_temp
                r_min = k
    return r_min


In [5]:
def pivoting(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    # escale pivot row min to 1.0
    tableau[row] = tableau[row]/tableau[row, col]
    # pivot proccess: convert al column to zero except row
    for k in range(len(tableau)):
        if k != row:
            tableau[k] = tableau[k] - tableau[k, col]*tableau[row, :]
    return tableau


In [6]:
def correct_negative_cost(tableau: np.ndarray, basic_var: list[int]):
    # correct first row of tableau because the artificial_var and min=-max position is negative
    for index, col in enumerate(basic_var):
        if tableau[0, col] < 0:
            # select the row with 1 because the rest are zeros
            row = index + 1
            tableau[0] = tableau[0] - tableau[0, col] * tableau[row, :]
    return tableau


In [7]:
def optimal_solution(tableau: np.ndarray, basic_var: list[int], artificial_var: list[int]=[]) -> None:
    # only work with simplex method because m = len(basic_var) and n = len(non_basic_var)
    dim_1, dim_2 = tableau.shape
    m = len(basic_var)
    n = dim_2 - dim_1 - len(artificial_var)  # for Big-M and Two-Phases
    x = np.zeros(m + n)
    x[basic_var] = tableau[1:, -1]
    print("Optimal solution: x_opt =", x[:n])


### Simplex Algorithm

In [8]:
def simplex(tableau: np.ndarray, basic_var: list[int]) ->np.ndarray:
    print(f"x_B = {basic_var}\n{tableau}")
    while (tableau[0, :-1] > 0).any():
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test
        r_min = minimum_ratio_test(col=tableau[1:, c_max], b=tableau[1:, -1]) + 1
        # pivoting
        tableau = pivoting(tableau, r_min, c_max)
        # swap row with col
        basic_var[r_min - 1] = c_max
        print(f"x_B = {basic_var}\n{tableau}")
    return tableau


### Big-M
The big-M procedure, each such constraint $i$ is augmented, together with its slack variable, with a so-called artificial variable $u_i$,  and the objective function is augmented with $−Mu_i$, where $M$ is a big positive real number. For big values of $M$ the simplex algorithm will put highest priority on making the value of the factor $Mu_i$ as small as possible, thereby setting the value of $u_i$ equal to zero. Big-M and two-phases are used when $0$ isn't feasible basic solution.

In [9]:
def big_M(tableau: np.ndarray, basic_var: list[int]) -> np.ndarray:
    print(f"x_B = {basic_var}\n{tableau}")
    # correct first row of tableau for each artificial var because the artificial_var position is -M
    tableau = correct_negative_cost(tableau, basic_var)
    # solve with simplex
    return simplex(tableau, basic_var)


### Two-Phases
The two-phase procedure add, in the same manner as employed in the big-M procedure, but instead of adding each artificial variable to the objetive with a large negative coefficient, the objetive function is replace by minus the sum of all artificial variables . During the **first phase**, the simplex algorithm tries to maximize this objetive, effectively trying to give all artificial variables the value zero.  By doing this a feasible basic solution is founded. When this has happened, the **second-phase** starts by replacing the artificial objetive function by the objetive function of the original model and solve this with simplex method.

In [10]:
def two_phases(tableau: np.ndarray, c: np.ndarray, basic_var: list[int], artificial_var: list[int]):
    print(f"x_B = {basic_var}\n{tableau}")
    # correct first row of tableau because the artificial_var position is -1
    tableau = correct_negative_cost(tableau, basic_var)
    print(f"x_B = {basic_var}\n{tableau}")

    print("Phase One")
    # while until each artifical var be negative and not be basic var
    while any(((u in basic_var) or (tableau[0, u] > 0)) for u in artificial_var):
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test
        r_min = minimum_ratio_test(col=tableau[1:, c_max], b=tableau[1:, -1]) + 1
        # pivoting
        tableau = pivoting(tableau, r_min, c_max)
        # swap row with col
        basic_var[r_min - 1] = c_max
        print(f"x_B = {basic_var}\n{tableau}")

    print("Phase Two")
    # delete the artificial variable because we found a solution factible
    tableau = np.delete(tableau, artificial_var, axis=1)
    # put c in first row of tableau
    tableau[0, :len(c)] = c
    # solve with simplex
    print(f"x_B = {basic_var}\n{tableau}")
    tableau = correct_negative_cost(tableau, basic_var)
    return simplex(tableau, basic_var)


### Book: Linear and Integer Optimization


#### Dovetail Model 
\begin{align*}
\max  3 x_{1}&+2 x_{2}& \\
\text { s.t. } \quad x_{1}&+x_{2}& \leq 9 \\
3 x_{1}&+x_{2}& \leq 18 \\
x_{1}&{} {}& \leq 7 \\
{}& \quad x_{2} & \leq 6 \\
x_{1}&, x_{2} \geq 0
\end{align*}

In [11]:
A = [[1, 1, 1, 0, 0, 0],
     [3, 1, 0, 1, 0, 0],
     [1, 0, 0, 0, 1, 0],
     [0, 1, 0, 0, 0, 1.0]]

b = [9, 18, 7, 6.]

c = [3, 2, 0, 0, 0, 0.]

basic_var = [2, 3, 4, 5]

tableau = build_tableau(A, b, c)
tableau = simplex(tableau, basic_var)
optimal_solution(tableau, basic_var)


x_B = [2, 3, 4, 5]
[[ 3.  2.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  6.]]
x_B = [2, 0, 4, 5]
[[  0.           1.           0.          -1.           0.           0.         -18.        ]
 [  0.           0.66666667   1.          -0.33333333   0.           0.           3.        ]
 [  1.           0.33333333   0.           0.33333333   0.           0.           6.        ]
 [  0.          -0.33333333   0.          -0.33333333   1.           0.           1.        ]
 [  0.           1.           0.           0.           0.           1.           6.        ]]
x_B = [1, 0, 4, 5]
[[  0.    0.   -1.5  -0.5   0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    1.5]]
Optimal solution: x_opt = [4.5 4.5]


**Two-Phases**: Dovetail Model modified: Constraint added $x_1 + x_2 \ge 5$

In [13]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "x6", 6: "x7", 7: "u1"}

tableau = np.array([[0, 0, 0, 0, 0, 0, 0, -1, 0],
                    [1, 1, 1, 0, 0, 0, 0, 0, 9.],
                    [3, 1, 0, 1, 0, 0, 0, 0, 18],
                    [1, 0, 0, 0, 1, 0, 0, 0, 7.],
                    [0, 1, 0, 0, 0, 1, 0, 0, 6.],
                    [1, 1, 0, 0, 0, 0, -1, 1, 5]])

c = [3, 2, 0, 0, 0, 0, 0.]

basic_var = [2, 3, 4, 5, 7]
artificial_var = [7]

tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [2, 3, 4, 5, 7]
[[ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
x_B = [2, 3, 4, 5, 7]
[[ 1.  1.  0.  0.  0.  0. -1.  0.  5.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
Phase One
x_B = [2, 3, 4, 5, 0]
[[ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  1.  0.  0.  0.  1. -1.  4.]
 [ 0. -2.  0.  1.  0.  0.  3. -3.  3.]
 [ 0. -1.  0.  0.  1.  0.  1. -1.  2.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
Phase Two
x_B = [2, 3, 4, 5, 0]
[[ 3.  2.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  1.  4.]
 [ 0. -2.  0.  1.  0.  0.  3.  3.]
 [ 0. -1.  0.  0.  1.  0.  1.  2.]
 [ 0.  1.  0.  0.  0.  1.  0.  6.]
 [ 1.  1.  0

**BIG-M**:  Ejemplo visto el ciclo pasado Lineal 6 

In [15]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "u1", 6: "u2"}
M = 10
A = [[2, -1, -1, 0, 0, 1, 0],
     [-1, 2, 0, -1, 0, 0, 1],
     [1, 1., 0., 0, 1, 0, 0.]]

b = [4, 2, 12.]

c = [2, 1, 0, 0, 0, -M, -M]

basic_var = [5, 6, 4]
artificial_var = [5, 6]

tableau = build_tableau(A, b, c)
tableau = big_M(tableau, basic_var)
optimal_solution(tableau, basic_var, artificial_var)


x_B = [5, 6, 4]
[[  2.   1.   0.   0.   0. -10. -10.   0.]
 [  2.  -1.  -1.   0.   0.   1.   0.   4.]
 [ -1.   2.   0.  -1.   0.   0.   1.   2.]
 [  1.   1.   0.   0.   1.   0.   0.  12.]]
x_B = [5, 6, 4]
[[ 12.  11. -10. -10.   0.   0.   0.  60.]
 [  2.  -1.  -1.   0.   0.   1.   0.   4.]
 [ -1.   2.   0.  -1.   0.   0.   1.   2.]
 [  1.   1.   0.   0.   1.   0.   0.  12.]]
x_B = [0, 6, 4]
[[  0.   17.   -4.  -10.    0.   -6.    0.   36. ]
 [  1.   -0.5  -0.5   0.    0.    0.5   0.    2. ]
 [  0.    1.5  -0.5  -1.    0.    0.5   1.    4. ]
 [  0.    1.5   0.5   0.    1.   -0.5   0.   10. ]]
x_B = [0, 1, 4]
[[  0.           0.           1.66666667   1.33333333   0.         -11.66666667 -11.33333333  -9.33333333]
 [  1.           0.          -0.66666667  -0.33333333   0.           0.66666667   0.33333333   3.33333333]
 [  0.           1.          -0.33333333  -0.66666667   0.           0.33333333   0.66666667   2.66666667]
 [  0.           0.           1.           1.           1.      

#### Equality constraints
Example 3.8.1. Consideer the following LO-model 

\begin{align*}
\max  40 x_{1}& + 100 x_{2}  + 150 x_{3}  \\
\text { s.t. }\quad x_{1} & + 2 x_{2}  +2 x_{3} =3 \\
 30 x_{1}& + 10 x_{2}  + 20 x_{3} =75 \\
 x_{1}&, x_{2}, x_{3} \geq 0
\end{align*}

#### More examples
Method 1: Big-M


In [17]:
M = 100
A = [[1, 2, 2, 1, 0],
     [30, 10, 20, 0, 1.]]

b = [3, 75.]
c = [40, 100, 150., -M, -M]

basic_var = [3, 4]

tableau = build_tableau(A, b, c)
tableau = big_M(tableau, basic_var)
optimal_solution(tableau, basic_var)

x_B = [3, 4]
[[  40.  100.  150. -100. -100.    0.]
 [   1.    2.    2.    1.    0.    3.]
 [  30.   10.   20.    0.    1.   75.]]
x_B = [3, 4]
[[3140. 1300. 2350.    0.    0. 7800.]
 [   1.    2.    2.    1.    0.    3.]
 [  30.   10.   20.    0.    1.   75.]]
x_B = [3, 0]
[[   0.          253.33333333  256.66666667    0.         -104.66666667  -50.        ]
 [   0.            1.66666667    1.33333333    1.           -0.03333333    0.5       ]
 [   1.            0.33333333    0.66666667    0.            0.03333333    2.5       ]]
x_B = [2, 0]
[[   0.     -67.5      0.    -192.5    -98.25  -146.25 ]
 [   0.       1.25     1.       0.75    -0.025    0.375]
 [   1.      -0.5      0.      -0.5      0.05     2.25 ]]
Optimal solution: x_opt = [2.25  0.    0.375]


Method 2: Simplex

In [19]:
A = [[1, 2, 2],
     [30, 10, 20.]]

b = [3, 75.]

c = [40, 100, 150.]

basic_var = [0, 1]

tableau = build_tableau(A, b, c)
print(f"x_B = {basic_var}\n{tableau}")

# correcting the coefficients different from zero in the basic var
tableau = pivoting(tableau, 1, 0)
tableau = pivoting(tableau, 2, 1)
print(f"x_B = {basic_var}\n{tableau}")

# simplex proccess
tableau = simplex(tableau, basic_var)

# Optimal solution for equality constraints
x = np.zeros_like(c)
x[basic_var] = tableau[1:,-1]
print("Optimal solution: x_opt =", x)


x_B = [0, 1]
[[ 40. 100. 150.   0.]
 [  1.   2.   2.   3.]
 [ 30.  10.  20.  75.]]
x_B = [0, 1]
[[   0.     0.    54.  -126. ]
 [   1.     0.     0.4    2.4]
 [  -0.     1.     0.8    0.3]]
x_B = [0, 1]
[[   0.     0.    54.  -126. ]
 [   1.     0.     0.4    2.4]
 [  -0.     1.     0.8    0.3]]
x_B = [0, 2]
[[   0.     -67.5      0.    -146.25 ]
 [   1.      -0.5      0.       2.25 ]
 [  -0.       1.25     1.       0.375]]
Optimal solution: x_opt = [2.25  0.    0.375]


### Cuarta Dirigida: Curso: Programación Lineal y flujo en redes

2. Hallar una solución aplicando las tablas del método simplex

In [21]:
A = [[2, 1, 1, 0, 0],
     [1, 0, 0, 1, 0],
     [0, 1, 0, 0, 1.]]

b = [9, 4, 3.]

c = [4, 5, 0, 0, 0.]

basic_var = [2, 3, 4]

tableau = build_tableau(A, b, c)
tableau = simplex(tableau, basic_var)
optimal_solution(tableau, basic_var)


x_B = [2, 3, 4]
[[4. 5. 0. 0. 0. 0.]
 [2. 1. 1. 0. 0. 9.]
 [1. 0. 0. 1. 0. 4.]
 [0. 1. 0. 0. 1. 3.]]
x_B = [2, 3, 1]
[[  4.   0.   0.   0.  -5. -15.]
 [  2.   0.   1.   0.  -1.   6.]
 [  1.   0.   0.   1.   0.   4.]
 [  0.   1.   0.   0.   1.   3.]]
x_B = [0, 3, 1]
[[  0.    0.   -2.    0.   -3.  -27. ]
 [  1.    0.    0.5   0.   -0.5   3. ]
 [  0.    0.   -0.5   1.    0.5   1. ]
 [  0.    1.    0.    0.    1.    3. ]]
Optimal solution: x_opt = [3. 3.]


Usando 2 fases

In [23]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "u1"}
A = [[1., 1., 1, 0, 0, 0],
     [-2, -1, 0, 1, 0, 0],
     [0., 1., 0, 0, -1, 1]]

b = [9, 4, 3.]

c = [4, 5, 0, 0.0, 0.0]
c_artificial = [0., 0., 0, 0, 0, -1]

basic_var = [2, 3, 5]
artificial_var = [5]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [2, 3, 5]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [-2. -1.  0.  1.  0.  0.  4.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
x_B = [2, 3, 5]
[[ 0.  1.  0.  0. -1.  0.  3.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [-2. -1.  0.  1.  0.  0.  4.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
Phase One
x_B = [2, 3, 1]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  0.  1.  0.  1. -1.  6.]
 [-2.  0.  0.  1. -1.  1.  7.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
Phase Two
x_B = [2, 3, 1]
[[ 4.  5.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  1.  6.]
 [-2.  0.  0.  1. -1.  7.]
 [ 0.  1.  0.  0. -1.  3.]]
x_B = [2, 3, 1]
[[ 4.  5.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  1.  6.]
 [-2.  0.  0.  1. -1.  7.]
 [ 0.  1.  0.  0. -1.  3.]]
x_B = [2, 3, 1]
[[  4.   0.   0.   0.   5. -15.]
 [  1.   0.   1.   0.   1.   6.]
 [ -2.   0.   0.   1.  -1.   7.]
 [  0.   1.   0.   0.  -1.   3.]]
x_B = [4, 3, 1]
[[ -1.   0.  -5.   0.   0. -45.]
 [  1.   0.   1.   0.   1.   6.]
 [ -1.   0.   1.   1.   0.  13.]
 [  1.   1.   1.   0.   0.   9.]]
Optimal s

Usando Big-M

In [25]:
M = 10

A = [[1., 1., 1, 0, 0, 0],
     [-2, -1, 0, 1, 0, 0],
     [0., 1., 0, 0, -1, 1]]

b = [9, 4, 3.]

c = [4, 5, 0, 0, 0, -M]

basic_var = [2, 3, 5]
artificial_var = [5]

tableau = build_tableau(A, b, c)
tableau = big_M(tableau, basic_var)
optimal_solution(tableau, basic_var, artificial_var)


x_B = [2, 3, 5]
[[  4.   5.   0.   0.   0. -10.   0.]
 [  1.   1.   1.   0.   0.   0.   9.]
 [ -2.  -1.   0.   1.   0.   0.   4.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
x_B = [2, 3, 5]
[[  4.  15.   0.   0. -10.   0.  30.]
 [  1.   1.   1.   0.   0.   0.   9.]
 [ -2.  -1.   0.   1.   0.   0.   4.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
x_B = [2, 3, 1]
[[  4.   0.   0.   0.   5. -15. -15.]
 [  1.   0.   1.   0.   1.  -1.   6.]
 [ -2.   0.   0.   1.  -1.   1.   7.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
x_B = [4, 3, 1]
[[ -1.   0.  -5.   0.   0. -10. -45.]
 [  1.   0.   1.   0.   1.  -1.   6.]
 [ -1.   0.   1.   1.   0.   0.  13.]
 [  1.   1.   1.   0.   0.   0.   9.]]
Optimal solution: x_opt = [0. 9.]


6. Resolver por el método de dos faces el siguiente LO

In [27]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "u1", 5: "u2"}
A = [[4., 2, -1, 0, 1, 0],
     [1., 4., 0, -1, 0, 1]]

b = [12, 6.0]

c_artificial = [0., 0., 0, 0, -1, -1]
c = [-2, -3., 0, 0]

basic_var = [4, 5]
artificial_var = [4, 5]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [4, 5]
[[ 0.  0.  0.  0. -1. -1.  0.]
 [ 4.  2. -1.  0.  1.  0. 12.]
 [ 1.  4.  0. -1.  0.  1.  6.]]
x_B = [4, 5]
[[ 5.  6. -1. -1.  0.  0. 18.]
 [ 4.  2. -1.  0.  1.  0. 12.]
 [ 1.  4.  0. -1.  0.  1.  6.]]
Phase One
x_B = [4, 1]
[[ 3.5   0.   -1.    0.5   0.   -1.5   9.  ]
 [ 3.5   0.   -1.    0.5   1.   -0.5   9.  ]
 [ 0.25  1.    0.   -0.25  0.    0.25  1.5 ]]
x_B = [0, 1]
[[ 0.          0.          0.          0.         -1.         -1.          0.        ]
 [ 1.          0.         -0.28571429  0.14285714  0.28571429 -0.14285714  2.57142857]
 [ 0.          1.          0.07142857 -0.28571429 -0.07142857  0.28571429  0.85714286]]
Phase Two
x_B = [0, 1]
[[-2.         -3.          0.          0.          0.        ]
 [ 1.          0.         -0.28571429  0.14285714  2.57142857]
 [ 0.          1.          0.07142857 -0.28571429  0.85714286]]
x_B = [0, 1]
[[ 0.          0.         -0.35714286 -0.57142857  7.71428571]
 [ 1.          0.         -0.28571429  0.14285714  2.57142857]


In [29]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "u1"}
A = [[1., 1, 1, 0., 0],
     [-1, 2, 0, -1, 1]]

b = [10, 2.0]

c_artificial = [0., 0., 0, 0, -1]
c = [2, 2., 0, 0]

basic_var = [2, 4]
artificial_var = [4]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [2, 4]
[[ 0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0. 10.]
 [-1.  2.  0. -1.  1.  2.]]
x_B = [2, 4]
[[-1.  2.  0. -1.  0.  2.]
 [ 1.  1.  1.  0.  0. 10.]
 [-1.  2.  0. -1.  1.  2.]]
Phase One
x_B = [2, 1]
[[ 0.   0.   0.   0.  -1.   0. ]
 [ 1.5  0.   1.   0.5 -0.5  9. ]
 [-0.5  1.   0.  -0.5  0.5  1. ]]
Phase Two
x_B = [2, 1]
[[ 2.   2.   0.   0.   0. ]
 [ 1.5  0.   1.   0.5  9. ]
 [-0.5  1.   0.  -0.5  1. ]]
x_B = [2, 1]
[[ 2.   2.   0.   0.   0. ]
 [ 1.5  0.   1.   0.5  9. ]
 [-0.5  1.   0.  -0.5  1. ]]
x_B = [0, 1]
[[  0.           2.          -1.33333333  -0.66666667 -12.        ]
 [  1.           0.           0.66666667   0.33333333   6.        ]
 [  0.           1.           0.33333333  -0.33333333   4.        ]]
x_B = [0, 1]
[[  0.           0.          -2.           0.         -20.        ]
 [  1.           0.           0.66666667   0.33333333   6.        ]
 [  0.           1.           0.33333333  -0.33333333   4.        ]]
x_B = [3, 1]
[[ -0.   0.  -2.   0. -20.]
 [  

9. Resolver por el método de dos faces el siguiente LO

In [31]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "u1"}
A = [[1, 1, 1, 1., 0, 0.],
     [4, 1, 6, 0, -1, 1]]

b = [1, 2.]

c_artificial = [0., 0., 0, 0, 0, -1]
c = [3, 3., 0, 0, 0.0]

basic_var = [2, 4]
artificial_var = [4]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c, basic_var, artificial_var)
optimal_solution(tableau, basic_var)


x_B = [2, 4]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  1.  0.  0.  1.]
 [ 4.  1.  6.  0. -1.  1.  2.]]
x_B = [2, 4]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  1.  0.  0.  1.]
 [ 4.  1.  6.  0. -1.  1.  2.]]
Phase One
x_B = [2, 0]
[[ 0.    0.    0.    0.    0.   -1.    0.  ]
 [ 0.    0.75 -0.5   1.    0.25 -0.25  0.5 ]
 [ 1.    0.25  1.5   0.   -0.25  0.25  0.5 ]]
Phase Two
x_B = [2, 0]
[[ 3.    3.    0.    0.    0.    0.  ]
 [ 0.    0.75 -0.5   1.   -0.25  0.5 ]
 [ 1.    0.25  1.5   0.    0.25  0.5 ]]
x_B = [2, 0]
[[ 3.    3.    0.    0.    0.    0.  ]
 [ 0.    0.75 -0.5   1.   -0.25  0.5 ]
 [ 1.    0.25  1.5   0.    0.25  0.5 ]]
x_B = [2, 0]
[[ 0.    2.25 -4.5   0.   -0.75 -1.5 ]
 [ 0.    0.75 -0.5   1.   -0.25  0.5 ]
 [ 1.    0.25  1.5   0.    0.25  0.5 ]]
x_B = [1, 0]
[[ 0.          0.         -3.         -3.          0.         -3.        ]
 [ 0.          1.         -0.66666667  1.33333333 -0.33333333  0.66666667]
 [ 1.          0.          1.66666667 -0.33333333  0.33333333

11. Elaborar un programa que al ingresar una $4 \times 6$ matriz arbitraria, verifique si dicha matriz representa las entradas de una tabla simplex (con 2 variables no básicas y 3 variables básicas). Testearlo con una matriz que si represente una tabla simplex y luego con una matrix de igual entradas que el anterior a excepción de la entrada (1, 6).

In [33]:
def check_tableau(tableau: np.ndarray, basic_var: list[int]):
    for index, col in enumerate(basic_var):
        # cost of basic variable is 0.0
        if not tableau[0, col] == 0.0:
            print("Basic cost isn't zero")
            return False
        else:
            for row in range(1, len(basic_var) + 1):
                if row != index + 1:
                    if not tableau[row, col] == 0.0:
                        print(f"Basic column {col} in position {row} isn't zero")
                        return False
                else:
                    if not tableau[row, col] == 1.0:
                        print(f"Basic column {col} in position {row} isn't one")
                        return False
        # b positive
        if not tableau[index + 1, -1] >= 0.0:
            print("B^{-1} b is negative")
            return False
    print("If it is a simplex table.")
    return True


In [34]:
tableau = np.array([[3., 2., 0., 0., 0., 0., 0.],
                    [1., 1., 1., 0., 0., 0., 9.],
                    [3., 1., 0., 1., 0., 0., 18.],
                    [1., 0., 0., 0., 1., 0., 7.],
                    [0., 1., 0., 0., 0., 1., 6.]])

basic_var = [2, 3, 4, 5]
check_tableau(tableau, basic_var);


If it is a simplex table.
